In [ ]:
from collections import defaultdict, UserList, UserDict
from collections.abc import Sequence, Iterable, Set
from collections import Counter
from itertools import chain, groupby, islice, product
from operator import itemgetter as itg
from functools import reduce
from tqdm import tqdm

import numpy as np
#from intbitset import intbitset as set

from line_profiler import LineProfiler
#profile = LineProfiler()

class PauliTerm:
    def __init__(self, pathfinder, n_qubits, paulis):
        self.path = [ memoryview(pathfinder[bit]) for bit in range(n_qubits) ]
        xyzs = np.zeros(n_qubits, dtype=int).tolist()
        for p in paulis:
            bit = p>>2
            xyzs[bit] = p&3
        self.xyzs = xyzs
        
    def __repr__(self):
        return str(bytes(seed[xyz] for seed,xyz in zip(self.path, self.xyzs)))
        
    def __hash__(self):
        return bytes(seed[xyz] for seed,xyz in zip(self.path, self.xyzs)).__hash__()

    def __str__(self):
        return f"//Term {self.xyzs}//"

    def __eq__(self, other):
        return self.__hash__() == other.__hash__()


#@profile
class SymPauliNoCoef:
    """Encode PauliXYZ in integer. 
    pauli = qubit<<2 | xyz , which xyz={0,1,2,3}:={i,x,y,z}

    Notes:
        64bit integer encoding allows up to 4.6e18 qubits.
        ((1<<64)-1)>>2 = 4,611,686,018,427,387,903
    """

    def __init__(self, n_qubits):
        #print(f"Initiating SymPauli...")
        self.n_qubits = n_qubits
        self.coefs = []
        self.pauliterms = []
        self.data = defaultdict(complex)
        self.pathfinder = [
                bytearray([0,1,2,3])
                for qubit in range(n_qubits)
                ]

    def __str__(self):
        xyz = ('I', 'X', 'Y', 'Z')
        qstr = []
        qstr = '\n'.join(qstr)
        return (
            "=================================\n"
            "----    SymPauli Overview    ----\n\n"
            f"{len(self.data)      = }\n"
            f"{self.coefs              = }\n"
            f"{self.pauliterms         = }\n"
            #f"self.coeffinder         = \n"
            f"{qstr}\n"
            "=================================\n"
             )

    def __getitem__(self, ref_id):
        return self.pauliterms[ref_id]

    def __len__(self):
        return self.data.__len__()

    def __add__(self, args):
        coef, *paulis = args
        #print(f"Adding {coef, paulis = }")
        n_qubits = self.n_qubits
        ref_id = len(self.pauliterms)
        self.coefs.append(coef)
        term = PauliTerm(self.pathfinder, n_qubits, paulis)
        self.pauliterms.append(term)
        self.data[term] += coef
        return self

    def renew_pathfinder(self):
        pathfinder = self.pathfinder
        for bit in range(self.n_qubits):
            q = memoryview(pathfinder[bit])
            for xyz in range(4):
                q[xyz] = xyz

    def eye(self):
        self += 1, *[n<<2 for n in range(self.n_qubits)]
        return self

    #==========================================================================
    #@profile
    def __imul__(self, args):
        coef, *paulis = args
        #print(f"Multiplying {coef, paulis = }")
        #print(f"{self.coefs = }")
        pathfinder = self.pathfinder
        for p in paulis:
            bit, xyz = p>>2, p&3
            #print(f"{p, bit, xyz = }")
            if xyz:
                q = memoryview(pathfinder[bit])
                if xyz == 1:
                    #print(f"\tReduced to X{bit}")
                    q[0], q[1], q[2], q[3] = q[1], q[0], q[3], q[2]
                elif xyz == 2:
                    #print(f"  Reduced to Y{bit}")
                    q[0], q[1], q[2], q[3] = q[2], q[3], q[0], q[1]
                elif xyz == 3:
                    #print(f"  Reduced to Z{bit}")
                    q[0], q[1], q[2], q[3] = q[3], q[2], q[1], q[0]
        return self

    def done_multiply(self):
        """Reset to Backend to Original State"""
        self.data = dict(zip(self.pauliterms, self.coefs))
        self.renew_pathfinder()
        return self


In [ ]:
class PauliTerm:
    def __init__(self, pathfinder, n_qubits, paulis):
        self.path = [ memoryview(pathfinder[bit]) for bit in range(n_qubits) ]
        xyzs = np.zeros(n_qubits, dtype=int).tolist()
        for p in paulis:
            bit = p>>2
            xyzs[bit] = p&3
        self.xyzs = xyzs
        
    def __repr__(self):
        return str(bytes(seed[xyz] for seed,xyz in zip(self.path, self.xyzs)))
        
    def __hash__(self):
        return bytes(seed[xyz] for seed,xyz in zip(self.path, self.xyzs)).__hash__()

    def __str__(self):
        return f"//Term {self.xyzs}//"

    def __eq__(self, other):
        return self.__hash__() == other.__hash__()



class SymPauliWithCoef:
    """Encode PauliXYZ in integer. 
    pauli = qubit<<2 | xyz , which xyz={0,1,2,3}:={i,x,y,z}

    Notes:
        64bit integer encoding allows up to 4.6e18 qubits.
        ((1<<64)-1)>>2 = 4,611,686,018,427,387,903
    """

    def __init__(self, n_qubits):
        #print(f"Initiating SymPauli...")
        self.n_qubits = n_qubits
        self.coefs = []
        self.pauliterms = []
        self.data = defaultdict(complex)
        self.clw = []
        self.coeffinder = [
                [ [], [], [], [] ]
                for qubit in range(n_qubits)
                ]
        self.pathfinder = [
                bytearray([0,1,2,3])
                for qubit in range(n_qubits)
                ]

    def __str__(self):
        xyz = ('I', 'X', 'Y', 'Z')
        qstr = []
        for bit,k in enumerate(self.coeffinder):
            qstr.append(f"  qubit = {bit}")
            for j,s in enumerate(k):
                qstr.append(f"    {xyz[j]} {s}")
        qstr = '\n'.join(qstr)
        return (
            "=================================\n"
            "----    SymPauli Overview    ----\n\n"
            f"{len(self.coefs)      = }\n"
            f"{self.coefs              = }\n"
            f"{self.pauliterms         = }\n"
            f"self.coeffinder         = \n"
            f"{qstr}\n"
            "=================================\n"
             )

    def __getitem__(self, ref_id):
        return self.pauliterms[ref_id]

    def __len__(self):
        return self.allterms.__len__()

    def __add__(self, args):
        coef, *paulis = args
        #print(f"Adding {coef, paulis = }")
        n_qubits = self.n_qubits
        coeffinder = self.coeffinder
        ref_id = len(self.pauliterms)
        self.clw.append(0)
        self.coefs.append(coef)
        term = PauliTerm(self.pathfinder, n_qubits, paulis)
        self.pauliterms.append(term)
        self.data[term] += coef
        for p in paulis:
            bit, xyz = p>>2, p&3
            if xyz:
                coeffinder[bit][xyz].append(ref_id)
        return self

    def renew_pathfinder(self):
        pathfinder = self.pathfinder
        for bit in range(self.n_qubits):
            q = memoryview(pathfinder[bit])
            for xyz in range(4):
                q[xyz] = xyz

    def eye(self):
        self += 1, *[n<<2 for n in range(self.n_qubits)]
        return self
    
    #==========================================================================
    def __imul__(self, args):
        coef, *paulis = args
        #print(f"Multiplying {coef, paulis = }")
        pathfinder = self.pathfinder
        coeffinder = self.coeffinder
        clw = np.zeros_like(self.coefs, dtype=np.int32)
        for p in paulis:
            bit, xyz = p>>2, p&3
            #print(f"{p, bit, xyz = }")
            if xyz:
                q = memoryview(pathfinder[bit])
                c = coeffinder[bit]
                if xyz == 1:
                    #print(f"\tReduced to X{bit}")
                    clw[c[2]] -= 1
                    clw[c[3]] += 1
                    q[0], q[1], q[2], q[3] = q[1], q[0], q[3], q[2]
                    c[0], c[1], c[2], c[3] = c[1], c[0], c[3], c[2]

                elif xyz == 2:
                    #print(f"  Reduced to Y{bit}")
                    clw[c[3]] -= 1
                    clw[c[1]] += 1
                    q[0], q[1], q[2], q[3] = q[2], q[3], q[0], q[1]
                    c[0], c[1], c[2], c[3] = c[2], c[3], c[0], c[1]

                elif xyz == 3:
                    #print(f"  Reduced to Z{bit}")
                    q[0], q[1], q[2], q[3] = q[3], q[2], q[1], q[0]
                    c[0], c[1], c[2], c[3] = c[3], c[2], c[1], c[0]
                    clw[c[1]] -= 1
                    clw[c[2]] += 1
        coefs = np.array(self.coefs, dtype=np.complex64)
        coefs *= (1j)**(clw%4)
        coefs *= coef
        self.coefs = coefs.tolist()
        return self

    def done_multiply(self):
        """Reset to Backend to Original State"""
        self.data = dict(zip(self.pauliterms, self.coefs))
        self.renew_pathfinder()
        return self

In [ ]:
# Simple show case of how to uses

H = SymPauliWithCoef(2)
H += 2, 2, 6  # 2 * I0
print(H.data.items())
H *= 1, 1, 5  # 1 * Y0 Y1
print(H.data.items())
H.done_multiply()
print(H.data.items())

In [ ]:
# Simple show case of how to uses

H = SymPauliNoCoef(2)
H += 2, 2, 6  # 2 * I0
print(H.data.items())
H *= 1, 1, 5  # 1 * Y0 Y1
print(H.data.items())
H.done_multiply()
print(H.data.items())

In [ ]:
# Benchmark for chain multiplication

import numpy as np

size = 60000
n_qubits = 40
np.random.seed = 100

testP = np.random.randint(0, 4, (size, n_qubits), dtype='uint8')
testC = np.ones(size)

from openfermion import QubitOperator as QO
control_data = []
for i,(paulis, coef) in enumerate(zip(testP, testC)):
    term = tuple( (i, chr(87+p)) for i,p in enumerate(paulis) if p )
    control_data.append(QO(term, coef))
#print(control_data[:10])

import numpy as np
experiment_data = []
for i, (coef, paulis) in enumerate(zip(testC, testP)):
    experiment_data.append([coef, *[i<<2 | p for i,p in enumerate(paulis)]])
#print(experiment_data[:10])

In [ ]:
%%time

control_init = QO((), 1)
for paulis in tqdm(control_data[:int(size*0.5)]):
    control_init += paulis

for _ in range(1):
    for paulis in tqdm(control_data[-int(size*0.5):]):
        control_init *= paulis


In [ ]:
%%time

experiment_init = SymPauliNoCoef(n_qubits).eye()
for paulis in tqdm(experiment_data[:-int(size*0.5)]):
    experiment_init += paulis
    
count = 0
for k,v in experiment_init.data.items():
    print(k.__repr__(), v)
    count += 1
    if count > 4:
        break
        
for _ in range(1):
    for paulis in tqdm(experiment_data[-int(size*0.5):]):
        experiment_init *= paulis

count = 0
for k,v in experiment_init.data.items():
    print(k.__repr__(), v)
    count += 1
    if count > 4:
        break

In [ ]:
%%time

experiment_init = SymPauliWithCoef(n_qubits).eye()
for paulis in tqdm(experiment_data[:-int(size*0.5)]):
    experiment_init += paulis

for _ in range(1):
    for paulis in tqdm(experiment_data[-int(size*0.5):]):
        experiment_init *= paulis


In [ ]:
# Benchmark for chain multiplication

from time import perf_counter as tt
from time import sleep
import numpy as np
from openfermion import QubitOperator as QO
np.random.seed = 100

def bench_jw_ori(control_data, n_qubits):
    sleep(0.1)
    t0 = tt()
    control_init = QO((), 1)
    for paulis in control_data[:int(size*0.5)]:
        control_init += paulis
    return tt()-t0

def bench_jw_new(experiment_data, n_qubits):
    sleep(0.1)
    t0 = tt()
    experiment_init = SymPauliNoCoef(n_qubits).eye()
    for paulis in experiment_data[:-int(size*0.5)]:
        experiment_init += paulis
    return tt()-t0


sizes = [2**2, 2**4, 2**6, 2**8, 2**10, 2**12, 2^13]
n_qubits = [1, 2, 4, 8, 12, 24, 28]

labels = list(zip(sizes, n_qubits))
limit = -1

old_timelog = []
new_timelog = []

r = 1

for n_q, size in zip(n_qubits[:limit], sizes[:limit]):

    testP = np.random.randint(0, 4, (size, n_q), dtype='uint8')
    testC = np.ones(size)

    control_data = []
    for i,(paulis, coef) in enumerate(zip(testP, testC)):
        term = tuple( (i, chr(87+p)) for i,p in enumerate(paulis) if p )
        control_data.append(QO(term, coef))
    experiment_data = []
    for i, (coef, paulis) in enumerate(zip(testC, testP)):
        experiment_data.append([coef, *[i<<2 | p for i,p in enumerate(paulis)]])

    ori_log = [bench_jw_ori(control_data, n_q) for _ in range(r)]
    new_log = [bench_jw_new(experiment_data, n_q) for _ in range(r)]

    for dt in ori_log:
        old_timelog.append((n_q, dt))
    for dt in new_log:
        new_timelog.append((n_q, dt))

old_t, new_t = np.array(old_timelog), np.array(new_timelog)

import pandas as pd

columns = ['n_qubit', 'runtime']
old_tpd = pd.DataFrame(old_t, columns=columns)
print(old_tpd)

new_tpd = pd.DataFrame(new_t, columns=columns)
new_tpd['speedup'] = -1 * pd.Series( new_tpd['runtime'] / old_tpd['runtime'] )
print(new_tpd)

In [ ]:
import matplotlib.pyplot as plt
import datetime

plt.figure(figsize=(18, 9))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

ax1 = plt.subplot(1, 2, 1)
ax1.set_yscale('log')
ax1.plot(old_tpd['n_qubit'], old_tpd['runtime'],
        c='k', label='OpenFermion', marker="o", markerfacecolor="w")
ax1.plot(new_tpd['n_qubit'], new_tpd['runtime'],
        c='b', label='SymPauli', marker="o", markerfacecolor="w")

plt.xlabel('size, n_qubit', size=16)
plt.ylabel('runtime (second)', size=16)
plt.xticks(n_qubits[:limit], labels[:limit], size=10)
plt.legend(loc='lower right')
plt.grid()
plt.title("Runtime (summation part only)", size=24)

ax2 = plt.subplot(1, 2, 2)
ax2.set_yscale('linear')
ax2.plot(new_tpd['n_qubit'], new_tpd['speedup'],
        c='b', label='SymPauli', marker="o", markerfacecolor="w")

plt.xlabel('size, n_qubit', size=16)
plt.ylabel('speedup (-OpenFermion / SymPauli)', size=16)
plt.xticks(n_qubits[:limit], labels[:limit], size=10)
plt.yticks(range(0, -10, -1), range(0, -10, -1))
plt.legend(loc='lower right')
plt.grid()
plt.title("Speedup", size=24)

plt.savefig(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M"))
plt.show()

In [ ]:
# Benchmark for chain multiplication

from time import perf_counter as tt
from time import sleep
import numpy as np
from openfermion import QubitOperator as QO
np.random.seed = 100

def bench_jw_new(experiment_data, n_qubits):
    sleep(0.1)
    experiment_init = SymPauliNoCoef(n_qubits).eye()
    for paulis in experiment_data[:-int(size*0.5)]:
        experiment_init += paulis
    t0 = tt()
    for _ in range(1):
        for paulis in experiment_data[-int(size*0.5):]:
            experiment_init *= paulis
    return tt()-t0


sizes = [2**2, 2**4, 2**6, 2**8, 2**10, 2**12, 2**13, 2**14, 2**15, 2**16, 2*817]
n_qubits = [1, 2, 4, 8, 12, 24, 28, 32, 36, 40, 44]

labels = list(zip(sizes, n_qubits))
limit = -1

old_timelog = []
new_timelog = []

r = 1

for n_q, size in zip(n_qubits[:limit], sizes[:limit]):

    testP = np.random.randint(0, 4, (size, n_q), dtype='uint8')
    testC = np.ones(size)

    experiment_data = []
    for i, (coef, paulis) in enumerate(zip(testC, testP)):
        experiment_data.append([coef, *[i<<2 | p for i,p in enumerate(paulis)]])

    new_log = [bench_jw_new(experiment_data, n_q) for _ in range(r)]

    for dt in new_log:
        new_timelog.append((size, dt))

new_t = np.array(new_timelog)

In [ ]:
import pandas as pd

new_tpd = pd.DataFrame(new_t, columns=columns)
print(new_tpd)

import matplotlib.pyplot as plt
import datetime

plt.figure(figsize=(9, 9))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

ax1 = plt.subplot(1, 1, 1)
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.plot(new_tpd['n_qubit'], new_tpd['runtime'],
        c='b', label='SymPauli', marker="o", markerfacecolor="w")

plt.xlabel('size', size=16)
plt.ylabel('runtime (second)', size=16)
#plt.xticks(n_qubits[:limit], sizes[:limit], size=10)
plt.legend(loc='lower right')
plt.grid()
plt.title("Runtime (SymPauli chain multiplication part only)", size=24)

plt.savefig(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M"))
plt.show()

In [ ]:
# Benchmark for chain multiplication (benchmark multiplication part only)

from time import perf_counter as tt
from time import sleep
import numpy as np
from openfermion import QubitOperator as QO
np.random.seed = 100

def bench_jw_ori(control_data, n_qubits):
    sleep(0.1)
    control_init = QO((), 1)
    for paulis in control_data[:int(size*0.5)]:
        control_init += paulis
    t0 = tt()
    for _ in range(1):
        for paulis in control_data[-int(size*0.5):]:
            control_init *= paulis
    return tt()-t0

def bench_jw_new(experiment_data, n_qubits):
    sleep(0.1)
    experiment_init = SymPauliNoCoef(n_qubits).eye()
    for paulis in experiment_data[:-int(size*0.5)]:
        experiment_init += paulis
    t0 = tt()
    for _ in range(1):
        for paulis in experiment_data[-int(size*0.5):]:
            experiment_init *= paulis
    return tt()-t0


sizes = [2**2, 2**4, 2**6, 2**8, 2**10, 2**12, 2^13]
n_qubits = [1, 2, 4, 8, 12, 24, 28]

labels = list(zip(sizes, n_qubits))
limit = -1

old_timelog = []
new_timelog = []

r = 1

for n_q, size in zip(n_qubits[:limit], sizes[:limit]):

    testP = np.random.randint(0, 4, (size, n_q), dtype='uint8')
    testC = np.ones(size)

    control_data = []
    for i,(paulis, coef) in enumerate(zip(testP, testC)):
        term = tuple( (i, chr(87+p)) for i,p in enumerate(paulis) if p )
        control_data.append(QO(term, coef))
    experiment_data = []
    for i, (coef, paulis) in enumerate(zip(testC, testP)):
        experiment_data.append([coef, *[i<<2 | p for i,p in enumerate(paulis)]])

    ori_log = [bench_jw_ori(control_data, n_q) for _ in range(r)]
    new_log = [bench_jw_new(experiment_data, n_q) for _ in range(r)]

    for dt in ori_log:
        old_timelog.append((n_q, dt))
    for dt in new_log:
        new_timelog.append((n_q, dt))

old_t, new_t = np.array(old_timelog), np.array(new_timelog)


In [ ]:
import pandas as pd

columns = ['n_qubit', 'runtime']
old_tpd = pd.DataFrame(old_t, columns=columns)
print(old_tpd)

new_tpd = pd.DataFrame(new_t, columns=columns)
new_tpd['speedup'] = pd.Series( old_tpd['runtime'] / new_tpd['runtime'] )
print(new_tpd)

import matplotlib.pyplot as plt
import datetime

plt.figure(figsize=(18, 9))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

ax1 = plt.subplot(1, 2, 1)
ax1.set_yscale('log')
ax1.plot(old_tpd['n_qubit'], old_tpd['runtime'],
        c='k', label='OpenFermion', marker="o", markerfacecolor="w")
ax1.plot(new_tpd['n_qubit'], new_tpd['runtime'],
        c='b', label='SymPauli', marker="o", markerfacecolor="w")

plt.xlabel('size, n_qubit', size=16)
plt.ylabel('runtime (second)', size=16)
plt.xticks(n_qubits[:limit], labels[:limit], size=10)
plt.legend(loc='lower right')
plt.grid()
plt.title("Runtime (chain multiplication part only)", size=24)

ax2 = plt.subplot(1, 2, 2)
ax2.set_yscale('log')
ax2.plot(new_tpd['n_qubit'], new_tpd['speedup'],
        c='b', label='SymPauli', marker="o", markerfacecolor="w")

plt.xlabel('size, n_qubit', size=16)
plt.ylabel('speedup (SymPauli / Openfermion)', size=16)
plt.xticks(n_qubits[:limit], labels[:limit], size=10)
plt.yticks(range(1, 2202, 400), range(1, 2202, 400))
plt.legend(loc='lower right')
plt.grid()
plt.title("Speedup", size=24)

plt.savefig(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M"))
plt.show()